In [ ]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Bidirectional, LSTM, GRU, RNN
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, CSVLogger
import numpy as np
import matplotlib.pyplot as plt

### 文本情感识别模型；

In [ ]:
#模型输入为shape = (50, 100)的词向量；
#数据类型为np.array

#序贯式模型；
model = Sequential()
#调节的参数为units, 这里设置的是units = 128;
model.add(Bidirectional(LSTM(units = 128, input_shape = (50, 100))))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Dense(3, activation = 'softmax'))

#定义一个优化器；
opt = SGD(lr = 0.001, momentum= 0.9)

#训练提前停止，当损失值连续10轮没有下降；
earlystoppping = EarlyStopping(monitor= 'loss', patience= 10, verbose= 1)

#保存训练最佳模型；
#best_checkpoint = str(5) + 'best.h5'
#modelcheckpoint = ModelCheckpoint(best_checkpoint, monitor= 'val_acc', save_best_only= True, 
                                  #mode= 'max', verbose= 1)

#记录训练中每轮的acc及loss，并写入csv中；
#csv_path = str(5) + '.csv'
#csv_log = CSVLogger(csv_path)

#这里优化器选择的是'adam',使用默认参数；
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics= ['accuracy'])
model.fit(train_X, train_Y, epochs = 100, validation_data = (test_X, test_Y), 
          batch_size = 128, verbose = 1,
          callbacks= [earlystoppping])

y_pre = model.predict(test_X, batch_size= 128)

#保存模型；
model.save(r'txt.h5')

#输出测试集中的指标值，包括acc(需升级keras至最新版本才会有这一值), precision, recall, F1;
#digits为精确到小数点后几位；
print(classification_report(test_Y.argmax(axis= 1), y_pre.argmax(axis= 1), digits= 5))

### 绘制训练过程中的损失值曲线；

In [ ]:
#绘制准确率的训练曲线同理，history为字典；
plt.plot(history.history['loss'])

plt.title('Train loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.legend(['Train', 'Test'])
plt.show()